In [1]:
import cv2
import matplotlib.pyplot as plt
import numpy as np
import tkinter as tk
from PIL import Image, ImageTk,ImageEnhance
from tkinter import filedialog,Toplevel
from tkinter import Menu

In [ ]:
# Global variables to store the original and edited images
original_img = None
edited_img = None
current_r = 100
current_g = 100
current_b = 100
current_contrast = 100

# Function to open and display an image
def open_image():
    global original_img, edited_img,current_g,current_b,current_contrast,current_r
    current_r = 100
    current_g = 100
    current_b = 100
    current_contrast = 100
    # Open a file dialog to select an image file
    file_path = filedialog.askopenfilename(filetypes=[("Image files", "*.jpg;*.jpeg;*.png;*.bmp;*.gif")])
    if file_path:
        # Open the image
        original_img = Image.open(file_path)
        edited_img = original_img  # Copy original image as the initial edited image

        # Resize the images to fit the frames
        display_image(original_img, original_label)

# Function to display an image in a specific label
def display_image(image, label):
    resized_img = image.resize((450, 450))
    img_tk = ImageTk.PhotoImage(resized_img)
    label.config(image=img_tk)
    label.image = img_tk  # Keep a reference to avoid garbage collection


# Function to update the edited image based on RGB values from sliders
def update_color(r, g, b):
    global original_img, edited_img, current_r, current_g, current_b
    
    if edited_img:
        # Cập nhật giá trị RGB hiện tại
        
        # Điều chỉnh các kênh màu
        r_factor = r / current_r
        g_factor = g / current_g
        b_factor = b / current_b

        # Tách hình ảnh thành các kênh màu R, G, B
        r_img, g_img, b_img = edited_img.split()

        # Áp dụng các thay đổi dựa trên giá trị RGB đã lưu
        r_img = r_img.point(lambda i: i * r_factor)
        g_img = g_img.point(lambda i: i * g_factor)
        b_img = b_img.point(lambda i: i * b_factor)

        # Gộp các kênh màu và hiển thị
        edited_img = Image.merge("RGB", (r_img, g_img, b_img))
        current_r = r
        current_g = g
        current_b = b
        display_image(edited_img, edited_label)

# Function to open the color adjustment window
def open_color_window():
    color_window = Toplevel(root)
    color_window.title("Adjust Colors")
    color_window.geometry("300x300")

    # Variables to track RGB values from scales
    r_value = tk.IntVar(value=current_r)
    g_value = tk.IntVar(value=current_g)
    b_value = tk.IntVar(value=current_b)

    # Red Scale
    r_scale = tk.Scale(color_window, from_=10, to=200, orient="horizontal", label="Red", variable=r_value)
    r_scale.pack()
    # Green Scale
    g_scale = tk.Scale(color_window, from_=10, to=200, orient="horizontal", label="Green", variable=g_value)
    g_scale.pack()
    # Blue Scale
    b_scale = tk.Scale(color_window, from_=10, to=200, orient="horizontal", label="Blue", variable=b_value)
    b_scale.pack()
    
    # Update color when scale values change
    def on_scale_change(event=None):
        update_color(r_value.get(), g_value.get(), b_value.get())

    # Bind scale changes to color update
    r_scale.bind("<ButtonRelease-1>", on_scale_change)
    g_scale.bind("<ButtonRelease-1>", on_scale_change)
    b_scale.bind("<ButtonRelease-1>", on_scale_change)

def open_contrast_window():
    contrast_windown = Toplevel(root)
    contrast_windown.title("Contrast Window")
    contrast_windown.geometry("300x300")
    
    #Variable to keep track of contrast value
    contrast_value = tk.IntVar(value = current_contrast)
    
    # Contrast scale
    contrast_scale = tk.Scale(contrast_windown, from_=1, to = 200, orient="horizontal", label= "Contrast", variable= contrast_value)
    contrast_scale.pack()
    
    # Update color when scale values change
    def on_scale_change(event=None):
        update_contrast(contrast_value.get())

    # Bind scale changes to color update
    contrast_scale.bind("<ButtonRelease-1>", on_scale_change)
    
def update_contrast(contrast_value):
    global current_contrast,edited_img
    img_cv = np.array(edited_img)
    img_cv_abs = cv2.convertScaleAbs(img_cv, alpha= ( contrast_value/current_contrast), beta=0)
    edited_img = Image.fromarray(img_cv_abs)
    
    current_contrast = contrast_value
    
    display_image(edited_img,edited_label)
    #y = x * contrast_value; y' = y * contrast_value/current_contrast
# Create the main window
root = tk.Tk()
root.title("Scion Image")
root.geometry("1200x1200")


# Create the menu bar
menu_bar = Menu(root)

# Add "File" menu with "Open", "Edit Color", and "Exit" options
file_menu = Menu(menu_bar, tearoff=0)
file_menu.add_command(label="Open", command=open_image)

file_menu.add_command(label="Exit", command=root.quit)
menu_bar.add_cascade(label="File", menu=file_menu)

process_menu = Menu(menu_bar,tearoff=0)
process_menu.add_command(label="Edit Color", command=open_color_window)
process_menu.add_command(label="Contrast", command=open_contrast_window)
menu_bar.add_cascade(label="Process",menu=process_menu)

root.config(menu=menu_bar)

# Create frames for original and edited images
frame_original = tk.Frame(root, width=300, height=400, bg="white")
frame_original.pack(side="left", fill="both", expand=True)

frame_edited = tk.Frame(root, width=300, height=400, bg="white")
frame_edited.pack(side="right", fill="both", expand=True)

# Labels to display images in each frame
original_label = tk.Label(frame_original, bg="white")
original_label.place(x=100, y=100)

# Label to display text "Original Image" below the original image
original_text_label = tk.Label(frame_original, text="Original Image", bg="white", font=("Arial", 10))
original_text_label.place(x=275, y=600)

edited_label = tk.Label(frame_edited, bg="white")
edited_label.place(x=100, y=100)

# Label to display text "Edited Image" below the edited image
edited_text_label = tk.Label(frame_edited, text="Edited Image", bg="white", font=("Arial", 10))
edited_text_label.place(x=275, y=600)

# Start the main loop
root.mainloop()

# Random comment

In [ ]:
def invert_image(img):
    # Convert the image to a numpy array
    img_np = np.array(img)
    
    # Invert the image
    inverted_img_np = 255 - img_np
    
    # Convert the numpy array back to an image
    inverted_img = Image.fromarray(inverted_img_np)
    
    return inverted_img